1. Install the required packages

In [ ]:
pip install pymongo

2. Set up the mongo client where the hastags and output tweets are located.

In [ ]:
from pymongo import MongoClient

# MongoDB setup
client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection string
db = client['twitter_db']
collection = db['tweets']

3. Scrape hashags using snscrape

In [ ]:
pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime

def scrape_tweets(hashtag, limit=100):
    query = f"#{hashtag}"
    tweets = []

    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) >= limit:
            break
        
        tweet_data = {
            'date': tweet.date,
            'id': tweet.id,
            'content': tweet.content,
            'user': tweet.user.username,
            'hashtags': [tag.lower() for tag in tweet.hashtags],
            'likes': tweet.likeCount,
            'retweets': tweet.retweetCount
        }
        tweets.append(tweet_data)
    
    return tweets

hashtag = "python"  # Replace with your desired hashtag
limit = 100  # Number of tweets to scrape

print(f"Scraping tweets for #{hashtag}...")
tweets = scrape_tweets(hashtag, limit)

3. Scrape using ntscraper

In [4]:
pip install ntscraper

Note: you may need to restart the kernel to use updated packages.


In [10]:
from ntscraper import Nitter

scraper = Nitter(log_level=1, skip_instance_check=False)

terms = ["AAPL"]

tweets = scraper.get_tweets(terms, mode='hashtag', number=2)

print(tweets)



Testing instances: 100%|██████████| 77/77 [01:36<00:00,  1.26s/it]

16-Aug-24 20:18:25 - No instance specified, using random instance https://nitter.privacydev.net


16-Aug-24 20:18:25 - Empty page on https://nitter.privacydev.net
{'tweets': [], 'threads': []}


4. Twint


In [12]:
pip install twint

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/653.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/653.6 kB ? eta -:--:--
     ---------------- ----------------------- 262.1/653.6 kB ? eta -:--:--
     -------------------------------------- 653.6/653.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33945 sha256=79a781ea09b497fdd9c711b1112f65719f8ee551cc6bc14b39cf137830db3175
  Stored in directory: c:\users\m\appdata\local\pip\cache\wheels\e1\0d\0e\1384ada6f5ca0ec87cd8d48d107d69b782fb76b3a138fc3f8a
  Running setup.py clean for cchardet
  Created wheel for googletransx: filename=googletransx-2.4.2-py3-none-any.whl si

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\cchardet
      copying src\cchardet\version.py -> build\lib.win-amd64-cpython-311\cchardet
      copying src\cchardet\__init__.py -> build\lib.win-amd64-cpython-311\cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cchardet
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cchardet)


5. Store in DB

In [ ]:

def store_in_mongodb(tweets):
    try:
        result = collection.insert_many(tweets)
        print(f"Inserted {len(result.inserted_ids)} documents into MongoDB")
    except Exception as e:
        print(f"An error occurred while storing tweets: {e}")

if tweets:
    print(f"Scraped {len(tweets)} tweets. Storing in MongoDB...")
    store_in_mongodb(tweets)
else:
    print("No tweets found.")